# Airline Customer Feedback Analysis

### Business Understanding

The airline industry is highly competitive and customer service is a critical area that can influence customer loyalty and brand reputation. Timely and effective handling of customer feedback is crucial for maintaining service quality. Analyzing customer feedback can provide insights into common concerns and areas for improvement.

### Project Goals

The primary goal of this project is twofold:

1. **Sentiment Analysis**: To automatically determine the sentiment of customer comments. The sentiment can be:
   - **Negative (Complaint)**: Indicates the customer faced an issue or is dissatisfied.
   - **Positive (Congratulation)**: Reflects customer satisfaction or a positive experience.
   - **Neutral (Opinion)**: Represents a customer's opinion that is neither a direct complaint nor praise.

2. **Complaint Classification**: To classify customer feedback into categories for routing to the appropriate department. This classification will help in streamlining the process of addressing customer feedback efficiently. Example categories include:
   - **Luggage Issues**: For complaints or feedback related to damaged or lost luggage.
   - **Flight Delays**: For issues pertaining to delayed or cancelled flights.
   - **Billing Issues**: For concerns related to overcharges, refunds, or payment discrepancies.

The end product will be a model or a set of models that can accurately tag incoming customer comments with the correct sentiment and category, allowing the customer service team to promptly address and resolve the issues.

In [51]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import nltk
from nltk.corpus import stopwords
import os.path
import json

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import little_mallet_wrapper as lmw


# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline


# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

path_to_mallet = 'mallet-2.0.8/bin/mallet'  # CHANGE THIS TO YOUR MALLET PATH

In [52]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/gabot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/gabot/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [53]:
# Load data
data = pd.read_excel('output.xlsx', engine='openpyxl')

# Change typo in column name
data.rename(columns={'descripciom': 'descripcion'}, inplace=True)


In [54]:
# Get stopwords for both English and Spanish
english_stopwords = stopwords.words('english')
spanish_stopwords = stopwords.words('spanish')
all_stopwords = set(english_stopwords + spanish_stopwords)
# Add additional Spanish stopwords
unwanted_words = []
with open('list_stopwords.txt', 'r', encoding='utf-8') as file:
    new_stopwords = json.load(file)

def clean_text(text):
    # Convert to string in case there are non-string types
    text = str(text).lower()
    text = re.sub(r'<[^>]+>', '', text)  # Remove HTML-like tags: <anything>
    for word in unwanted_words:
        text = re.sub(r'\b' + re.escape(word) + r'\b', '', text) # Remove specific unwanted words
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-záéíóúñ\s]', '', text)  # Remove punctuation and special chars
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespaces

    # Remove stopwords for both English and Spanish
    text = ' '.join(word for word in text.split() if word not in all_stopwords)
    return text

# Apply the cleaning function to the 'descripcion' column
data['cleaned_descripcion'] = data['descripcion'].astype(str).apply(clean_text)
df = data

In [55]:
# Convert to list
data = data.cleaned_descripcion.values.tolist()

# Remove non-string elements
data = [str(sent) for sent in data]

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['gracias escribirnos recibido correo seguida daremos respuesta emergencias '
 'favor comunicate']


In [56]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['gracias', 'escribirnos', 'recibido', 'correo', 'seguida', 'daremos', 'respuesta', 'emergencias', 'favor', 'comunicate']]


In [57]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['gracias', 'escribirnos_recibido_correo', 'seguida_daremos_respuesta', 'emergencias', 'favor', 'comunicate']


In [58]:
# Define initial stop words sets
stop_words_en = set(stopwords.words('english'))
stop_words_sp = set(stopwords.words('spanish'))

# Add additional Spanish stopwords
new_stopwords = []
with open('list_stopwords.txt', 'r', encoding='utf-8') as file:
    new_stopwords = json.load(file)

# Combine all stopwords
all_stopwords = stop_words_en.union(stop_words_sp).union(set(new_stopwords))

# Define stopwords excluding certain words if needed
not_stopwords = set() # This should be a set of words you want to exclude from the stop words
final_stop_words = set([word for word in all_stopwords if word not in not_stopwords])

print(final_stop_words)

{'estarías', 'hasn', 'tenías', 'habremos', 'noches', 'esos', 'hubiera', "you've", 'una', 'ella', 'isn', 'up', 'sois', 'tenidas', 'png', 'ain', "needn't", 'solo', 'ourselves', 'suyos', 'now', 'tenidos', 'cid', "shan't", 'sentid', 'sus', 'uno', 'los', "you'll", "isn't", 'tenía', 'hayáis', 'estéis', 'correo electrónico', 'a', 'sin', 'esta', 'te', "doesn't", 'hubieses', 'muchas', 'formato', "wasn't", 'era', 'habrían', 'adjunto podrá', 'sea', 'seríais', 'favorito', 'sí', 'hubiéramos', 'where', 'vuestra', 'estábamos', 'ustede', 'most', "hasn't", 'but', 'gmail', 'esa', 'son', 'ls', 'qué', 'from', 'mis', 'de', 'contra', 'do', 'div', 'donde', 'estados', 'habré', 'otros', 'www', 'quedo', 'how', 'estarán', 'm', 'important', 'before', 'eran', 'feature', 'tiene', 'días', 'the', "weren't", 'porque', 'seré', 'tenga', 'cual', 'estado', 'suya', 'estuve', 'hn_evehu', 'siente', 'habríamos', 'mail_itinerary', 'having', 'which', 'by', 'tened', 'noche', 'tendréis', 'les', 'those', 'será', 'en', "mightn't", 

In [59]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    # return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    return [[word for word in simple_preprocess(str(doc)) if word not in final_stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [60]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [61]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_bigrams)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [62]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('comunicate', 1),
  ('emergencias', 1),
  ('escribirnos_recibido', 1),
  ('respuesta', 1),
  ('seguida_daremos', 1)]]

In [63]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [71]:
# Save LDA model
lda_model.save("lda_model")

# Optionally, you can also save the corpus and id2word dictionary for later use
# Save corpus
gensim.corpora.MmCorpus.serialize('corpus.mm', corpus)

# Save id2word dictionary
id2word.save('id2word.dict')

In [84]:
# Load LDA model
lda_model = gensim.models.ldamodel.LdaModel.load("lda_model")

# Load corpus
corpus = gensim.corpora.MmCorpus('corpus.mm')

# Load id2word dictionary
id2word = gensim.corpora.Dictionary.load('id2word.dict')

In [81]:
# Print the Keyword in the 4 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.040*"monterrey" + 0.037*"mexico" + 0.037*"julio" + 0.034*"ciudad" + '
  '0.028*"cancun" + 0.020*"enviado" + 0.016*"viaje" + 0.016*"pendiente" + '
  '0.014*"agosto" + 0.012*"guadalajara"'),
 (1,
  '0.045*"vuelo" + 0.016*"mxn" + 0.015*"reservacion" + 0.010*"horas" + '
  '0.010*"salida" + 0.009*"equipaje" + 0.009*"aeropuerto" + 0.008*"pasajeros" '
  '+ 0.008*"clave" + 0.007*"vuelos"'),
 (2,
  '0.041*"cotizacion" + 0.019*"viaje" + 0.011*"tarifa" + 0.011*"pago" + '
  '0.010*"informacion" + 0.009*"email" + 0.009*"aerobus" + 0.009*"fecha" + '
  '0.008*"servicio" + 0.008*"equipo"'),
 (3,
  '0.017*"datos" + 0.011*"maleta" + 0.010*"solicito" + 0.008*"espero" + '
  '0.008*"hora" + 0.007*"pueden" + 0.007*"momento" + 0.006*"cdmx" + '
  '0.005*"hrs" + 0.005*"vuelo"'),
 (4,
  '0.061*"flight" + 0.018*"please" + 0.017*"airport" + 0.015*"get" + '
  '0.014*"usd" + 0.012*"reservation" + 0.011*"need" + 0.011*"baggage" + '
  '0.010*"travel" + 0.010*"bag"')]


In [77]:
# Assuming `df['enhanced_cleaned_descripcion']` is your cleaned text column
texts1 = df['cleaned_descripcion'].apply(lambda x: x.split())

# Create a dictionary representation of the documents
dictionary = corpora.Dictionary(texts1)

# Filter out words that occur less frequently
dictionary.filter_extremes(no_below=5, no_above=0.5)

# Convert document into the bag-of-words (BoW) format = list of (token_id, token_count)
corpus = [dictionary.doc2bow(text) for text in texts1]

In [86]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster  Freq
topic                                           
0      0.101505  0.061858       1        1   NaN
1      0.203524 -0.058987       2        1   NaN
2      0.197590 -0.094508       3        1   NaN
3     -0.331459 -0.250064       4        1   NaN
4     -0.171161  0.341701       5        1   NaN, topic_info=                    Term  Freq  Total Category  logprob  loglift
0             comunicate   0.0    0.0  Default  30.0000     30.0
1            emergencias   0.0    0.0  Default  29.0000     29.0
2   escribirnos_recibido   0.0    0.0  Default  28.0000     28.0
3              respuesta   0.0    0.0  Default  27.0000     27.0
4        seguida_daremos   0.0    0.0  Default  26.0000     26.0
..                   ...   ...    ...      ...      ...      ...
25    excedente_residual   0.0    0.0   Topic5 -13.5216      NaN
26                 fecha   0.0    0.0   Topic5 -13.5187      NaN
27              generada   0.0    0.0   Topic5 -13.5216      NaN
28                genere   0.0    0.0   Topic5 -13.5187      NaN
29               hacerlo   0.0    0.0   Topic5 -13.5215      NaN

[180 rows x 6 columns], token_table=Empty DataFrame
Columns: [Topic, Freq, Term]
Index: [], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5])

K means

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english')

# Fit and transform the cleaned descriptions to a TF-IDF matrix
tfidf_matrix = vectorizer.fit_transform(df['cleaned_descripcion'])

In [69]:
# Choose the number of clusters
num_clusters = 5  # This is an arbitrary choice and should be optimized

# Initialize K-Means and fit it to the TF-IDF matrix
km = KMeans(n_clusters=num_clusters, random_state=42)
km.fit(tfidf_matrix)

# Assign the cluster labels to the DataFrame
clusters = km.labels_.tolist()
df['cluster'] = clusters

In [70]:
# Print top terms per cluster
print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
for i in range(num_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:  # replace 10 with n words per cluster
        print(' %s' % terms[ind], end='')
    print()

Top terms per cluster:
Cluster 0: bebé preguntar quería debe equipaje ser equivoco er eqvc equívoco
Cluster 1: noches buenas hola ola tal vuelo pregunta disculpe puedo quisiera
Cluster 2: image monterrey comprar méxico compra twitter ciudad forma blue cancún
Cluster 3: hola important blueaerolinea buenas tardes vuelo día buen gracias blue
Cluster 4: vuelo mxn horas salida si minutos tua tarifa aeropuerto podrás
